In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import expand_dims
import numpy.ma as ma
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as mtpltcm
from mpl_toolkits.mplot3d import Axes3D
import random
from netCDF4 import Dataset
import cartopy.crs as ccrs
from tensorflow import keras
from keras.layers import AveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import Add
from keras.models import load_model
import numpy.ma as ma
import os
import glob
np.set_printoptions(threshold=np.inf)

INPUTTEST=np.load("/kaggle/input/testfile/Inputfiletemptest.npy")
OUTPUTTEST=np.load("/kaggle/input/testfile/Highresfiletemptest.npy")
INPUTTRAIN=np.load("/kaggle/input/training-error/Inputfiletemp2017-2019.npy")
OUTPUTTRAIN=np.load("/kaggle/input/training-error/Highresfiletemp2017-2019.npy")
model = load_model("/kaggle/input/gen-epochs/TempSRgencomb19.0.keras")
print(INPUTTEST.shape, OUTPUTTEST.shape)
print(INPUTTRAIN.shape, OUTPUTTRAIN.shape)
RMSE=tf.keras.metrics.RootMeanSquaredError()

def RMSEERRORFUNC(models, INPUT, OUTPUT, RMSE):
    Error=[]
    for i in range(INPUT.shape[0]):
            gen=INPUT[i]
            era=OUTPUT[i]
            era=era*51
            era=np.reshape(era, (48,48))
            gen=np.reshape(gen, (1, 12, 12, 1))
            OUT = model.predict(gen, verbose=0)
            OUT=np.reshape(OUT,(48,48))
            OUT=OUT*51
            RMSE.update_state(OUT,era)
            Error.append(RMSE.result().numpy())
            RMSE.reset_state()
    RMSEMean=np.mean(np.array(Error))
    RMSESD=np.std(np.array(Error))
    return (RMSEMean, RMSESD)
    

(TrainingMean, TrainingSD)= RMSEERRORFUNC(model, INPUTTRAIN, OUTPUTTRAIN, RMSE)
print("Train")
print(TrainingMean, TrainingSD)
#np.save("TrainingMean",np.array(TrainingMean))
#np.save("TrainingSD",np.array(TrainingSD))
(TestMean, TestSD)= RMSEERRORFUNC(model, INPUTTEST, OUTPUTTEST, RMSE)
print("Test")
print(TestMean, TestSD)
#np.save("TestMean",np.array(TestMean))
#np.save("TestSD",np.array(TestSD)

